Using Content Based Filtering to recommend similar items by:

* Creating user-feature matrix
* Creating item-feature matrix
* Measuring similraity using dot product as metric
* Recommending top-k similar items

Second Approach

* Perform dimensionality reduction using PCA on user_feature and item_feature matrices

I used rows from transactions record

Limited customers to customers who bought at least two items

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [8]:
sale_path = "/content/drive/Shareddrives/Datathon/Datathon/InventoryAndSale_snapshot_data/Sales_snapshot_data/TT T06-2023_split_1.xlsx" # @param {type:"string"}

In [11]:
products_path = "/content/drive/Shareddrives/Datathon/Datathon/InventoryAndSale_snapshot_data/MasterData/Productmaster.xlsx" # @param {type:"string"}

In [12]:
sale_df = pd.read_excel(sale_path)

In [10]:
sale_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29772 entries, 0 to 29771
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   month                      29772 non-null  int64 
 1   week                       29772 non-null  int64 
 2   site                       29772 non-null  int64 
 3   branch_id                  29772 non-null  int64 
 4   channel_id                 29772 non-null  object
 5   distribution_channel       29772 non-null  object
 6   distribution_channel_code  29772 non-null  object
 7   sold_quantity              29772 non-null  int64 
 8   cost_price                 29772 non-null  int64 
 9   net_price                  29772 non-null  int64 
 10  customer_id                29772 non-null  object
 11  product_id                 29772 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.7+ MB


In [13]:
product_df = pd.read_excel(products_path)
product_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94867 entries, 0 to 94866
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            94867 non-null  int64  
 1   index                 94867 non-null  int64  
 2   color                 94822 non-null  object 
 3   color_group           94867 non-null  object 
 4   listing_price         94840 non-null  float64
 5   price_group           94840 non-null  object 
 6   gender                94867 non-null  object 
 7   product_group         94867 non-null  object 
 8   detail_product_group  94867 non-null  object 
 9   shoe_product          94867 non-null  object 
 10  size_group            94867 non-null  object 
 11  size                  94722 non-null  float64
 12  age_group             94867 non-null  object 
 13  activity_group        94867 non-null  object 
 14  image_copyright       94867 non-null  object 
 15  lifestyle_group    

In [15]:
users = sale_df
df = users.merge(product_df, on='product_id')
df.head()

,month,week,site,branch_id,channel_id,distribution_channel,distribution_channel_code,sold_quantity,cost_price_x,net_price,...,launch_season,mold_code,heel_height,code_lock,option,cost_price_y,product_syle_color,product_syle,brand_name,vendor_name
0,2023006,202322,1101,1100,CHTT,Bán lẻ,FP,1,316200,465000,...,23AP,NaN,NaN,NaN,P04-S01-H**,316200.0,1cdccfddebf04285a10c871fdf2f1ccfXNG,1cdccfddebf04285a10c871fdf2f1ccf,Brand1,vendor2
1,2023006,202325,1504,1500,CHTT,Bán lẻ,FP,1,726920,1069000,...,23AP,DEV078/DRB078,NaN,NaN,P06-C04-L**,679320.0,d468eb983cbb4849a54ed39610fb4e89XNH,d468eb983cbb4849a54ed39610fb4e89,Brand2,NaN
2,2023006,202324,1157,1100,CHTT,Bán lẻ,FP,1,726920,1069000,...,23AP,DEV078/DRB078,NaN,NaN,P06-C04-L**,679320.0,d468eb983cbb4849a54ed39610fb4e89XNH,d468eb983cbb4849a54ed39610fb4e89,Brand2,NaN
3,2023006,202322,1100,1100,TGPP,Hợp đồng,ZF1,1,726920,812440,...,23AP,DEV078/DRB078,NaN,NaN,P06-C04-L**,679320.0,d468eb983cbb4849a54ed39610fb4e89XNH,d468eb983cbb4849a54ed39610fb4e89,Brand2,NaN
4,2023006,202325,1511,1500,CHTT,Bán lẻ,FP,1,726920,1069000,...,23AP,DEV078/DRB078,NaN,NaN,P06-C04-L**,679320.0,d468eb983cbb4849a54ed39610fb4e89XNH,d468eb983cbb4849a54ed39610fb4e89,Brand2,NaN


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29772 entries, 0 to 29771
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   month                      29772 non-null  int64  
 1   week                       29772 non-null  int64  
 2   site                       29772 non-null  int64  
 3   branch_id                  29772 non-null  int64  
 4   channel_id                 29772 non-null  object 
 5   distribution_channel       29772 non-null  object 
 6   distribution_channel_code  29772 non-null  object 
 7   sold_quantity              29772 non-null  int64  
 8   cost_price_x               29772 non-null  int64  
 9   net_price                  29772 non-null  int64  
 10  customer_id                29772 non-null  object 
 11  product_id                 29772 non-null  object 
 12  Unnamed: 0                 29772 non-null  int64  
 13  index                      29772 non-null  int

In [17]:
feature_subset = ['color_group', 'price_group', 'detail_product_group', 'size_group', 'age_group', 'activity_group', 'lifestyle_group', 'brand_name']

In [18]:
df1 = df[['customer_id', 'product_id'] + feature_subset]
dummies_df = pd.get_dummies(df1, columns=feature_subset)
dummies_df

,customer_id,product_id,color_group_KHÁC,color_group_NHIỀU MÀU,color_group_SÁNG,color_group_TRẮNG,color_group_TỐI,color_group_ĐEN,color_group_ĐEN,price_group_100<200,...,activity_group_Thể thao,activity_group_Văn phòng,lifestyle_group_Casual,lifestyle_group_Fashion,lifestyle_group_Formal,lifestyle_group_Khác,lifestyle_group_Sport,brand_name_Brand1,brand_name_Brand2,brand_name_Brand3
0,dd5a95cfa,1cdccfddebf04285a10c871fdf2f1ccfXNG39,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
1,41fd17be1,d468eb983cbb4849a54ed39610fb4e89XNH44,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
2,f3d610794,d468eb983cbb4849a54ed39610fb4e89XNH44,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
3,19691a7e7,d468eb983cbb4849a54ed39610fb4e89XNH44,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
4,b8d51499a,d468eb983cbb4849a54ed39610fb4e89XNH44,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29767,c4b296c29,5b01dd75a4184d669b5b1accfe180139HOG28,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
29768,355c33838,6a5afaf87af7407ca5a817f8e9e0474cXAM39,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
29769,9654e071d,b39fdf889fff4625ba159641cd1ceeebHOG34,0,1,0,0,0,0,0,1,...,0,0,1,0,0,0,0,1,0,0
29770,81c0c4b70,c3d09fe213d64f1b954126d6ab70da3cHOG36,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0


In [20]:
minimum_items = 1
groupby_customer = dummies_df.groupby('customer_id')

l = []
cutomer_ids = []
article_ids = []
for key in groupby_customer.groups.keys():
    temp = groupby_customer.get_group(key)
    if temp.product_id.nunique() >= minimum_items:
        l.append(temp.drop('product_id', axis=1).sum(numeric_only=True).values)
        cutomer_ids.append(key)
        article_ids.extend(temp.product_id.values.tolist())

In [21]:
user_feature = pd.DataFrame(l, columns = dummies_df.columns[2:])
normalized_user_feature = user_feature.div(user_feature.sum(axis=1), axis=0)
normalized_user_feature.insert(0, 'customer_id', cutomer_ids)
normalized_user_feature = normalized_user_feature.set_index('customer_id')
normalized_user_feature

,color_group_KHÁC,color_group_NHIỀU MÀU,color_group_SÁNG,color_group_TRẮNG,color_group_TỐI,color_group_ĐEN,color_group_ĐEN,price_group_100<200,price_group_200<300,price_group_300<600,...,activity_group_Thể thao,activity_group_Văn phòng,lifestyle_group_Casual,lifestyle_group_Fashion,lifestyle_group_Formal,lifestyle_group_Khác,lifestyle_group_Sport,brand_name_Brand1,brand_name_Brand2,brand_name_Brand3
customer_id,,,,,,,,,,,,,,,,,,,,,
004c2ae83,0.000000,0.011905,0.041667,0.000000,0.059524,0.011905,0.0,0.000000,0.095238,0.017857,...,0.000000,0.000000,0.059524,0.0,0.000000,0.000000,0.065476,0.125000,0.000000,0.0
007e6bb72,0.000000,0.012500,0.012500,0.000000,0.062500,0.037500,0.0,0.000000,0.043750,0.081250,...,0.000000,0.000000,0.100000,0.0,0.000000,0.000000,0.025000,0.125000,0.000000,0.0
0086cd855,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.125000,0.000000,...,0.000000,0.000000,0.125000,0.0,0.000000,0.000000,0.000000,0.125000,0.000000,0.0
01347a866,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.0,0.000000,0.000000,0.125000,...,0.000000,0.000000,0.062500,0.0,0.000000,0.000000,0.062500,0.125000,0.000000,0.0
0196d459c,0.001866,0.007463,0.024254,0.000000,0.070896,0.020522,0.0,0.027985,0.011194,0.050373,...,0.000000,0.001866,0.072761,0.0,0.003731,0.003731,0.044776,0.100746,0.024254,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fca7cc257,0.003472,0.010417,0.010417,0.000000,0.086806,0.013889,0.0,0.090278,0.034722,0.000000,...,0.000000,0.000000,0.125000,0.0,0.000000,0.000000,0.000000,0.125000,0.000000,0.0
fd15d3401,0.003546,0.008865,0.035461,0.003546,0.059397,0.014184,0.0,0.035461,0.023936,0.027482,...,0.000887,0.000887,0.083333,0.0,0.000887,0.003546,0.037234,0.107270,0.017730,0.0
fe121da2d,0.000000,0.000000,0.104167,0.000000,0.020833,0.000000,0.0,0.041667,0.083333,0.000000,...,0.000000,0.000000,0.125000,0.0,0.000000,0.000000,0.000000,0.125000,0.000000,0.0


In [23]:
item_feature = dummies_df.drop_duplicates(subset='product_id')
item_feature = item_feature[item_feature.product_id.isin(article_ids)].drop('customer_id', axis=1)
item_feature = item_feature.set_index('product_id')
item_feature

,color_group_KHÁC,color_group_NHIỀU MÀU,color_group_SÁNG,color_group_TRẮNG,color_group_TỐI,color_group_ĐEN,color_group_ĐEN,price_group_100<200,price_group_200<300,price_group_300<600,...,activity_group_Thể thao,activity_group_Văn phòng,lifestyle_group_Casual,lifestyle_group_Fashion,lifestyle_group_Formal,lifestyle_group_Khác,lifestyle_group_Sport,brand_name_Brand1,brand_name_Brand2,brand_name_Brand3
product_id,,,,,,,,,,,,,,,,,,,,,
1cdccfddebf04285a10c871fdf2f1ccfXNG39,0,0,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,1,0,0
d468eb983cbb4849a54ed39610fb4e89XNH44,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
e29283acf2a84141b48a921cbdc46590XNH29,0,0,0,0,1,0,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0
c177bba7b83a4177b04954c5825cab8dXNH39,0,0,0,0,1,0,0,1,0,0,...,0,0,1,0,0,0,0,1,0,0
94c3f5fe7691442081f3ae9727a1eea7XAM41,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5b01dd75a4184d669b5b1accfe180139HOG28,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
6a5afaf87af7407ca5a817f8e9e0474cXAM39,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
b39fdf889fff4625ba159641cd1ceeebHOG34,0,1,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,1,0,0


In [24]:
scores = normalized_user_feature.dot(item_feature.T)
scores

product_id,1cdccfddebf04285a10c871fdf2f1ccfXNG39,d468eb983cbb4849a54ed39610fb4e89XNH44,e29283acf2a84141b48a921cbdc46590XNH29,c177bba7b83a4177b04954c5825cab8dXNH39,94c3f5fe7691442081f3ae9727a1eea7XAM41,27db4f7691c04b8d8955bfe837d898baHOG39,46a638cc5e394e3a83f356da64c11f37XDD35,03d6cf7934df449696b46ddd15873a5dDEN35,1e9d0fc8fe7642469651b2790c55d54fXNG34,c1519bc70149488398dee05e35f0959aDOO32,...,c24f9f371462492e918a033c41cdc30dXMN45,a8365c26749743eaaad871687828cdcdDEN41,15f276aed4b74dfba992af01ba789807DOO27,25f7928f8ff44fccbbfb84bcc4b69e51DEN39,316ce6085a0b4492bccb80749885c93bDEN23,5b01dd75a4184d669b5b1accfe180139HOG28,6a5afaf87af7407ca5a817f8e9e0474cXAM39,b39fdf889fff4625ba159641cd1ceeebHOG34,c3d09fe213d64f1b954126d6ab70da3cHOG36,191d4a6b67d449a5b1fe902e259d9654XAM43
customer_id,,,,,,,,,,,,,,,,,,,,,
004c2ae83,0.386905,0.261905,0.619048,0.410714,0.279762,0.654762,0.595238,0.470238,0.595238,0.785714,...,0.279762,0.458333,0.476190,0.011905,0.577381,0.583333,0.279762,0.565476,0.589286,0.505952
007e6bb72,0.518750,0.243750,0.537500,0.493750,0.293750,0.612500,0.650000,0.662500,0.493750,0.693750,...,0.293750,0.656250,0.437500,0.037500,0.612500,0.550000,0.293750,0.493750,0.575000,0.537500
0086cd855,0.625000,0.250000,0.750000,0.750000,0.250000,0.500000,0.625000,0.500000,0.500000,0.500000,...,0.250000,0.625000,0.375000,0.000000,0.375000,0.500000,0.250000,0.500000,0.375000,0.875000
01347a866,0.500000,0.281250,0.468750,0.500000,0.406250,0.531250,0.656250,0.656250,0.406250,0.656250,...,0.406250,0.625000,0.468750,0.000000,0.656250,0.468750,0.406250,0.406250,0.531250,0.500000
0196d459c,0.498134,0.363806,0.537313,0.569030,0.410448,0.399254,0.513060,0.501866,0.472015,0.477612,...,0.410448,0.516791,0.408582,0.091418,0.460821,0.468284,0.410448,0.455224,0.449627,0.552239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fca7cc257,0.597222,0.256944,0.736111,0.881944,0.333333,0.430556,0.625000,0.517361,0.597222,0.506944,...,0.333333,0.607639,0.388889,0.104167,0.562500,0.513889,0.333333,0.597222,0.388889,0.826389
fd15d3401,0.510638,0.356383,0.566489,0.609043,0.380319,0.434397,0.533688,0.492021,0.506206,0.476950,...,0.380319,0.514184,0.426418,0.086879,0.467199,0.491135,0.380319,0.479610,0.432624,0.597518
fe121da2d,0.500000,0.270833,0.562500,0.500000,0.187500,0.791667,0.604167,0.500000,0.687500,0.687500,...,0.187500,0.479167,0.500000,0.041667,0.625000,0.604167,0.187500,0.583333,0.604167,0.541667


In [26]:
def get_rcmnd(customer_id, scores):
    cutomer_scores = scores.loc[customer_id]
    customer_prev_items = groupby_customer.get_group(customer_id)['product_id']
    prev_dropped = cutomer_scores.drop(customer_prev_items.values)
    ordered = prev_dropped.sort_values(ascending=False)
    return ordered, customer_prev_items

In [33]:
k = 6
customer_id = scores.index[1]
rcmnds, prev_items = get_rcmnd(customer_id, scores)

In [34]:
rcmnds[:k]

product_id
729f721d1f01443299df69f330d6ba89XNH43    0.7375
f295954c7689427ea5760ebaaa238587XAM41    0.7375
c154f852c8dd4950bff08c3333aefb00NAU40    0.7375
55d51e2f4e91446a85d382ead2621a7cXAM39    0.7375
a0b6ce97a657447f886c8ac4b4481204XAM44    0.7375
60fd9083f8a04c97bc4bce0da74d059aXNH38    0.7375
Name: 007e6bb72, dtype: float64

In [30]:
prev_items

12014    3348e9605e7c401b9c607ce533319ae6DEN36
13568    fc7d9b4b6043488e8c8d0c1bca3ba619XNH43
14558    777d61c218ad46fa86b73e6f9a4b0345NAD41
19551    a1e6f85295f54f64b87eeb24eea4fd80XAM39
20467    58afed3f8b55487ea4288ab1a1bfad75KEM36
21422    39682ba437e347f58d6d8d05dfdc8e14XAM38
23726    1658dfa254ec4c968989f6e6e52924e1DEN42
26381    77aa7aef8bd24a2395a71369d8b925fdXAM34
26543    53fc812ce32542c08f8e07cb90f8c6e2DEN40
26868    43f34270db8547e4958f5c9ad49845bdTIM33
28797    be3e5f43ebc84bc68f88bcbd33022596DEN41
28898    83ba3a6de9374a3abf30dc0d9405fe48XNH32
29050    e9879dc57cd644518520c4bb4432d304XNH41
29265    dd46d5b96bb14654b0920c3d575a0020DEN39
29367    f857a9f78fb84cae90ee7c14544f745cDEN38
29526    c76201a77ff047bdab5fa4e2cb192f61XAM31
29563    2352863807f441a49e3d32df6ed40b19KEM37
29591    0208b6eb651e442598b9b0683bf5c3b9DEN30
29658    0845a4988bcc45f4957e16fb7c0dcf5eNAD43
29693    2f8de871e4f740bc9110834c7943e44cDEN39
Name: product_id, dtype: object